# exploration of the data

in this notebook we'll take a look at the gathered data from the MIMIC-III sql requests as well as the eICU sql requests.

This data is gathered to be able to train a ML model to be able to predict AKI


In [ ]:
# imports
import plotly.express as px
import pandas as pd
import numpy as np
import os
from pathlib import Path
import re
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import matplotlib.pyplot as plt
from aki_ml import *

In [ ]:
mimic_dir = Path(Path.cwd() / 'data' / 'mimiciii' )
mimic_query = Path(mimic_dir / 'queried')
mimic_prepro = Path(mimic_dir / 'preprocessed')
eicu_dir = Path(Path.cwd() / 'data' / 'eicu')
eicu_query = Path(eicu_dir / 'queried')
eicu_prepro = Path(eicu_dir / 'preprocessed')
mimic_dir

In [ ]:
# list contents and add them in a dict for easier processing:
def get_files(database_dir):
    files_dirs = [x for x in database_dir.glob('**/*.parquet') if x.is_file()]
    files_names = [x.stem for x in files_dirs]
    files = dict(zip(files_names, files_dirs))
    return files
files_mimic = get_files(mimic_query)
files_eicu = get_files(eicu_query)
files_mimic

In [ ]:
files_eicu

## Dataframes

We'll import the different dataset in pandas dataframes, to be able to visualise them easily:


TODO:

1. perhaps use cleanup_data on the dataframes, to check the content of the dataframes which is being handed to the ML model.
2. create images of this output 

In [ ]:
def create_dataframe_dict(parquetdict: dict):
    df_dict = dict()
    df_column_names = dict()
    for k,v in parquetdict.items():
        print(k,v)
        df_dict[k]=pd.read_parquet(v)
        # df_dict[k]=cleanup_data(v)
        df_column_names[k] = df_dict[k].columns
    return df_dict,df_column_names
mimiciii_df_dict, mimic_column_names = create_dataframe_dict(files_mimic)
eicu_df_dict, eicu_column_names = create_dataframe_dict(files_eicu)

## visualize parameters


### after postgres


In [ ]:
i="chart_vitals_stay"
j="diasbp_mean"
amount_of_data_mimic = mimiciii_df_dict[i][j].count()
amount_of_data_eicu = eicu_df_dict[i][j].count()
fig = make_subplots(rows=1, cols=2, subplot_titles=("mimic data amount: {}".format(amount_of_data_mimic),"eicu data, amount: {}".format(amount_of_data_eicu)))
fig.add_trace(go.Box(y=mimiciii_df_dict[i][j], name='mimic',boxpoints=False,), row=1, col=1)
fig.add_trace(go.Box(y=eicu_df_dict[i][j], name='eicu',boxpoints=False,), row=1, col=2)
fig.update_layout(
    yaxis_title=j,
    boxmode='group' # group together boxes of the different traces for each value of x
)
fig.show()

In [ ]:
def create_plots(df_mimic, df_eicu):
    x = 'icustay_id'
    columns_to_skip = ['hadm_id','icustay_id','charttime','subject_id','seq_num','admittime']
    root_dir = Path(Path.cwd() / 'data'/ 'comparison')
    try:
        root_dir.mkdir(parents=True, exist_ok=False)
    except FileExistsError:
        print("Folder is already there")
    else:
        print("Folder was created").mkdir(parents=True, exist_ok=True)
    
    for i in df_mimic:
        for j in df_mimic[i]:
            print(i,j)
            if j not in (columns_to_skip):
                if(i=='comorbidities'):
                    x='hadm_id'
                else:
                    x='icustay_id'
                if j not in df_mimic[i].columns or j not in df_eicu[i].columns:
                    print("skipping {}".format(j))
                    continue
                amount_of_data_mimic = df_mimic[i][j].count()
                amount_of_data_eicu = df_eicu[i][j].count()
                fig = make_subplots(rows=1, cols=2, subplot_titles=("mimic data amount: {}".format(amount_of_data_mimic),"eicu data, amount: {}".format(amount_of_data_eicu)))
                fig.add_trace(go.Box(y=df_mimic[i][j], name='mimic',boxpoints=False,), row=1, col=1)
                fig.add_trace(go.Box(y=df_eicu[i][j], name='eicu',boxpoints=False,), row=1, col=2)
                fig.update_layout(
                    yaxis_title=j,
                    boxmode='group' # group together boxes of the different traces for each value of x
                )
                # fig.write_html(root_dir / str(i+"_"+j+".html"))
                fig.write_image(root_dir / str(i+"_"+j+".jpeg"))
create_plots(mimiciii_df_dict, eicu_df_dict)

### After preprocessing


In [ ]:
eicu_prepro_files = get_files(eicu_prepro)
mimic_prepro_files = get_files(mimic_prepro)
# mimic_prepro_files['INFO_DATASET_7days_creatinine2']

In [ ]:
mimic_prepro_df_dict, mimic_prepro_column_names = create_dataframe_dict(mimic_prepro_files)
eicu_prepro_df_dict,eicu_prepro_column_names = create_dataframe_dict(eicu_prepro_files)

Let's check how much columns we have in each dataset:

In [ ]:
dataset = 'INFO_DATASET_7days_creatinine2'

print("mimic {} vs eicu {}".format(len(mimic_prepro_column_names[dataset]),len(eicu_prepro_column_names[dataset])))

Let's compare which columns we don't have in the eicu files:

In [ ]:
for i in mimic_prepro_column_names[dataset]:
    if i not in eicu_prepro_column_names[dataset]:
        print(i)

In [ ]:
def plot_categorical(df_mimic, df_eicu, parameter):
    fig, axes = plt.subplots(nrows=2, ncols=1,sharex=True)
    df_mimic.groupby(parameter)['ICUSTAY_ID',].count().plot(kind='bar', ax=axes[0])
    df_eicu.groupby(parameter)['ICUSTAY_ID',].count().plot(kind='bar', ax=axes[1])
    plt.show()
plot_categorical(mimic_prepro_df_dict['INFO_DATASET_7days_creatinine2'],eicu_prepro_df_dict['INFO_DATASET_7days_creatinine2'],'GENDER')

In [ ]:
def create_plots_prepro(df_mimic_dict:dict, df_eicu_dict:dict):
    x = 'icustay_id'
    columns_to_skip = ['hadm_id','icustay_id','charttime','subject_id','seq_num','admittime']
    root_dir = Path(Path.cwd() / 'data'/ 'comparison'/'preprocessing')
    try:
        root_dir.mkdir(parents=True, exist_ok=False)
    except FileExistsError:
        print("Folder is already there")
    else:
        print("Folder was created")
    
    for i in df_mimic_dict:
        # print(i)
        for j in df_mimic_dict[i]:
            print("{} ----> {}".format(i,j))
            if j not in (columns_to_skip):
                if(i=='comorbidities'):
                    x='hadm_id'
                else:
                    x='icustay_id'
                if j not in df_mimic_dict[i].columns or j not in df_eicu_dict[i].columns:
                    print("skipping {}".format(j))
                    continue
                amount_of_data_mimic = df_mimic_dict[i][j].count()
                amount_of_data_eicu = df_eicu_dict[i][j].count()
                fig = make_subplots(rows=1, cols=2, subplot_titles=("mimic data amount: {}".format(amount_of_data_mimic),"eicu data, amount: {}".format(amount_of_data_eicu)))
                fig.add_trace(go.Box(y=df_mimic_dict[i][j], name='mimic',boxpoints=False,), row=1, col=1)
                fig.add_trace(go.Box(y=df_eicu_dict[i][j], name='eicu',boxpoints=False,), row=1, col=2)
                fig.update_layout(
                    yaxis_title=j,
                    boxmode='group' # group together boxes of the different traces for each value of x
                )
                # fig.write_html(root_dir / str(i+"_"+j+".html"))
                fig.write_image(root_dir / str(i+"_"+j+".jpeg"))
create_plots_prepro(df_mimic_dict=mimic_prepro_df_dict, df_eicu_dict=eicu_prepro_df_dict)

In [ ]:
def create_plots_prepro_path(df_mimic_dict:dict, df_eicu_dict:dict):
    x = 'icustay_id'
    columns_to_skip = ['hadm_id','icustay_id','charttime','subject_id','seq_num','admittime']
    root_dir = Path(Path.cwd() / 'data'/ 'comparison'/'cleanup')
    try:
        root_dir.mkdir(parents=True, exist_ok=False)
    except FileExistsError:
        print("Folder is already there")
    else:
        print("Folder was created")
    
    for i in df_mimic_dict:
        print(i,df_mimic_dict[i], df_eicu_dict[i])
        df_mimic = cleanup_data(df_mimic_dict[i])
        df_eicu = cleanup_data(df_eicu_dict[i])
        for j in df_mimic:
            print("{} ----> {}".format(i,j))
            if(j=="DIASBP_MEAN"):
                print(df_eicu[["DIASBP_MEAN",'DIASBP_MAX','DIASBP_MIN']].head(10))
            if j not in (columns_to_skip):
                if(i=='comorbidities'):
                    x='hadm_id'
                else:
                    x='icustay_id'
                if j not in df_mimic.columns or j not in df_eicu.columns:
                    print("skipping {}".format(j))
                    continue
                amount_of_data_mimic = df_mimic[j].count()
                amount_of_data_eicu = df_eicu[j].count()
                fig = make_subplots(rows=1, cols=2, subplot_titles=("mimic data amount: {}".format(amount_of_data_mimic),"eicu data, amount: {}".format(amount_of_data_eicu)))
                fig.add_trace(go.Box(y=df_mimic[j], name='mimic',boxpoints=False,), row=1, col=1)
                fig.add_trace(go.Box(y=df_eicu[j], name='eicu',boxpoints=False,), row=1, col=2)
                fig.update_layout(
                    yaxis_title=j,
                    boxmode='group' # group together boxes of the different traces for each value of x
                )
                # fig.write_html(root_dir / str(i+"_"+j+".html"))
                fig.write_image(root_dir / str(i+"_"+j+".jpeg"))
create_plots_prepro_path(df_mimic_dict=mimic_prepro_files, df_eicu_dict=eicu_prepro_files)


### check the cleaned dataframes

these are fed to the AKI classification model.

In [ ]:
dataset = 'INFO_DATASET_7days_creatinine+urine2'
mimic_non_clean = mimic_prepro_df_dict[dataset]
eicu_non_clean = eicu_prepro_df_dict[dataset]
#not cleaned files
#we'll only count the files if they are larger than 1.0 as there are lot's of zeros due to NaN being converted to 0
amount_of_data_mimic_non_clean = mimic_non_clean[mimic_non_clean["DIASBP_MEAN"]>1.0]["DIASBP_MEAN"].count()
amount_of_data_eicu_non_clean = eicu_non_clean[eicu_non_clean["DIASBP_MEAN"]>1.0]["DIASBP_MEAN"].count()
#cleaned files
mimic_clean = cleanup_data(mimic_prepro_files[dataset])
eicu_clean = cleanup_data(eicu_prepro_files[dataset])
amount_of_data_mimic_clean = mimic_clean[mimic_clean["DIASBP_MEAN"]>1.0]["DIASBP_MEAN"].count()
amount_of_data_eicu_clean = eicu_clean[eicu_clean["DIASBP_MEAN"]>1.0]["DIASBP_MEAN"].count()

In [ ]:
fig = make_subplots(
    rows=2,
    cols=2,
    subplot_titles=(
        "mimic data amount before: {}".format(amount_of_data_mimic_non_clean),
        "eicu data amount before: {}".format(amount_of_data_eicu_non_clean),
        "mimic data amount: {}".format(amount_of_data_mimic_clean),
        "eicu data, amount: {}".format(amount_of_data_eicu_clean),
    ),
    shared_xaxes=True,
)

# before cleaning
fig.add_trace(
    go.Histogram(
        x=mimic_non_clean["DIASBP_MEAN"],
        name="mimic before cleaning",
    ),
    row=1,
    col=1,
)
fig.add_trace(
    go.Histogram(
        x=eicu_non_clean["DIASBP_MEAN"],
        name="eicu before cleaning",
    ),
    row=1,
    col=2,
)
# cleaned dataframes
fig.add_trace(go.Histogram(x=mimic_clean["DIASBP_MEAN"], name="mimic"), row=2, col=1)
fig.add_trace(go.Histogram(x=eicu_clean["DIASBP_MEAN"], name="eicu"), row=2, col=2)
fig.update_layout(
    yaxis_title="DIASBP_MEAN",
    boxmode="group",  # group together boxes of the different traces for each value of x
)
fig.update_xaxes(range=[0, 200])
fig.update_yaxes(range=[0, 1500])

fig.show()


## AUROC plots

we've created a csv which compares a model trained on a certain hospital to other hospitals.
We want to plot this with continuous error bars, so we'll first capture an array of the hospital results themselves. Then we'll calculate the quantiles to plot a continuous error bar

In [ ]:
auroc_local = pd.read_csv(Path(Path.cwd() / 'data'/ 'eicu' / 'auroc_local_matrix.csv'))

In [ ]:
auroc_line = dict()
for hospital in auroc_local.index:
    auroc_hospital = auroc_local.at[hospital, "hospital_{}".format(hospital)]
    auroc_line[hospital]=auroc_hospital
auroc_line_df = pd.DataFrame.from_dict(auroc_line, orient='index', columns=['AUROC'])
# auroc_line_df.columns='AUROC'
auroc_line_df.index.name = 'hospital_id'
px.line(auroc_line_df)

In [ ]:
i='hospital_58'
auroc_l_series = auroc_local[i].copy()

In [ ]:
auroc_l_series.drop(58)

In [ ]:
auroc_l_series = dict()
for i in auroc_local.columns:
    index_h=int(i.split('_')[1])
    aur_series = auroc_local[i].copy().drop(index=index_h)
    auroc_l_series[index_h] = aur_series.quantile([0.25,0.75])

In [ ]:
auroc_l_series_df = pd.DataFrame.from_dict(auroc_l_series)

In [ ]:
import plotly.graph_objs as go
fig = go.Figure(
    [
        go.Scatter(
        x=auroc_line_df.index,
        y=auroc_line_df.AUROC,
        name="hospitals auroc on itself"
    ),
        go.Scatter(
        x=list(auroc_l_series_df.loc[0.25].index) + list(auroc_l_series_df.loc[0.75].index)[::-1],  # x, then x reversed
        y=list(auroc_l_series_df.loc[0.25].values) + list(auroc_l_series_df.loc[0.75].values)[::-1],  # upper, then lower reversed
        fill='toself',
        fillcolor='rgba(0,100,80,0.2)',
        line=dict(color='rgba(255,255,255,0)'),
        hoverinfo="skip",
        name="hospital model on other hospitals, Q1-Q3"
        # showlegend=False
    )]
)
fig.update_layout(
    title="AKI model AUROC local",
    xaxis_title="hospital_id",
    yaxis_title="AUROC results"
)
fig.show()
fig.write_html(Path(Path.cwd() / 'data'/ 'eicu'/'local_auroc_line.html'))


In [ ]:
auroc_global = pd.read_csv(Path(Path.cwd() / 'data'/ 'eicu' / 'auroc_global_matrix.csv'), index_col=0)
auroc_combined = pd.read_csv(Path(Path.cwd() / 'data'/ 'eicu' / 'auroc_combined_matrix.csv'), index_col=0)


In [ ]:
fig = go.Figure([
    go.Line(
        x=auroc_global.index,
        y=auroc_global.eicu,
        name="global"
    ),
    go.Line(
        x=auroc_combined.index,
        y=auroc_combined.combined,
        name="combined"
    )
])
fig.update_layout(
    title="AKI model AUROC global",
    xaxis_title="hospital_id",
    yaxis_title="AUROC results"
)
fig.show()
fig.write_html(Path(Path.cwd() / 'data'/ 'eicu'/'auroc_lines.html'))

## plot insights

amount of people per category for the hospitals of train_id

In [ ]:
insights = pd.read_csv(Path(Path.cwd() / 'data/eicu/insights.csv'), index_col=0)
trained = pd.read_csv(Path(Path.cwd() / 'data/eicu/trained.csv'), header=None, names=['hospital_id','bool'])
#we're only interested in the hospitals on which we'll be training
trained_pos = trained[trained['bool']==True]['hospital_id']

we'll filter the insights dataset for those hospitals of trained_pos, plus we only care about the creat_only values

In [ ]:
insights_filtered = insights[insights['hospital'].isin(trained_pos)]

In [ ]:
insights_filtered

In [ ]:
fig = go.Figure([
    go.Scatter(
        x=insights_filtered.hospital,
        y=insights_filtered.creat_only_normal,
        name="normal"
    ),
    go.Scatter(
        x=insights_filtered.hospital,
        y=insights_filtered.creat_only_AKI_1,
        name="AKI 1"
    ),
    go.Scatter(
        x=insights_filtered.hospital,
        y=insights_filtered.creat_only_AKI_2,
        name="AKI 2"
    ),
    go.Scatter(
        x=insights_filtered.hospital,
        y=insights_filtered.creat_only_AKI_3,
        name="AKI 3"
    )
])
fig.update_layout(
    title="patients after cleaning, creatinine based only",
    xaxis_title="hospital_id",
    yaxis_title="amount"
)
fig.show()


## combined view of local and global plots, together with the amounts

In [ ]:
# GLOBAL
fig = make_subplots(rows=2, cols=1,shared_xaxes=True,subplot_titles=['auroc results','amount of patients per hospital per category'])

fig.append_trace(
    go.Scatter(
        x=auroc_global.index,
        y=auroc_global.eicu,
        name="global"
    ), row=1, col=1)

fig.append_trace(
    go.Scatter(
        x=auroc_combined.index,
        y=auroc_combined.combined,
        name="combined"
    ), row=1, col=1)

fig.append_trace(go.Scatter(
    x=insights_filtered.hospital,
    y=insights_filtered.creat_only_normal,
    name="normal"
), row=2, col=1)

fig.append_trace(go.Scatter(
    x=insights_filtered.hospital,
    y=insights_filtered.creat_only_AKI_1,
    name="AKI 1"
), row=2, col=1)
fig.append_trace(go.Scatter(
    x=insights_filtered.hospital,
    y=insights_filtered.creat_only_AKI_2,
    name="AKI 2"
), row=2, col=1)
fig.append_trace(go.Scatter(
    x=insights_filtered.hospital,
    y=insights_filtered.creat_only_AKI_3,
    name="AKI 3"
), row=2, col=1)
fig.show()


## compare data from a "good" scoring hospital with that from a "bad" one

In [ ]:
good = 197
bad = 262
good_df = pd.read_parquet(Path(Path.cwd() / 'data/eicu/hospital_{}/clean_data/INFO_DATASET_7days_creatinine2.parquet'.format(good)))
bad_df = pd.read_parquet(Path(Path.cwd() / 'data/eicu/hospital_{}/clean_data/INFO_DATASET_7days_creatinine2.parquet'.format(bad)))
mimic_df = pd.read_parquet(Path(Path.cwd() / 'data/mimiciii/mimic/clean_data/INFO_DATASET_7days_creatinine2.parquet'))
eicu_df = pd.read_parquet(Path(Path.cwd() / 'data/eicu/eicu/clean_data/INFO_DATASET_7days_creatinine2.parquet'))
good_df_desc = good_df.describe()
bad_df_desc = bad_df.describe()
mimic_df_desc = mimic_df.describe()
eicu_df_desc = eicu_df.describe()

In [ ]:
def plot_metric_comparison(metric):
    fig = make_subplots(rows=len(metric), cols=1,shared_xaxes=True,subplot_titles=(metric))
    for idx,i in enumerate(metric, start=1):
        fig.add_trace(go.Scatter(
            x=good_df_desc.columns,
            y=good_df_desc.loc['{}'.format(i)],
            name="good_df_{}".format(i)
        ), row=idx, col=1)
        fig.add_trace(go.Scatter(
            x=bad_df_desc.columns,
            y=bad_df_desc.loc['{}'.format(i)],
            name="bad_df_{}".format(i)
        ), row=idx, col=1)
        fig.add_trace(go.Scatter(
            x=mimic_df_desc.columns,
            y=mimic_df_desc.loc['{}'.format(i)],
            name="mimic_df_{}".format(i)
        ), row=idx, col=1)
        fig.add_trace(go.Scatter(
            x=eicu_df_desc.columns,
            y=eicu_df_desc.loc['{}'.format(i)],
            name="eicu_df_{}".format(i)
        ), row=idx, col=1)
    fig.update_layout(
        title="comparison of {} of values between a good, a bad and mimic dataframe (for training)".format(metric),
    )
    fig.show()
    fig.write_html(Path(Path.cwd() / 'data/eicu/insights_min_mean_max.html'))
plot_metric_comparison(['mean'])

In [ ]:
good_path = Path(Path.cwd(
) / 'data/eicu/hospital_{}/model/creatinine_model_hospital_{}/metrics/creatinine_model_hospital_{}_auroc_comparison.csv'.format(good, good, good))
good_auroc_df = pd.read_csv(good_path, index_col=0,
                            header=None, names=['from', 'to', 'auroc'])
good_auroc = good_auroc_df[good_auroc_df.at['hospital_{}'.format(
    good), 'to'] == "hospital_{}".format(good)].auroc

bad_path = Path(Path.cwd(
) / 'data/eicu/hospital_{}/model/creatinine_model_hospital_{}/metrics/creatinine_model_hospital_{}_auroc_comparison.csv'.format(bad, bad, bad))
bad_auroc_df = pd.read_csv(bad_path, index_col=0,
                           header=None, names=['from', 'to', 'auroc'])
bad_auroc = bad_auroc_df[bad_auroc_df.at['hospital_{}'.format(
    bad), 'to'] == "hospital_{}".format(bad)].auroc

eicu_auroc = 0.7255278928385078

mimic_auroc = 0.7564216218262945


In [ ]:
print("good: \t{} \t\t{}".format(good_df_desc.loc['count']['AKI'], good_auroc.values[0]))
print("bad: \t{} \t\t{}".format(bad_df_desc.loc['count']['AKI'], bad_auroc.values[0]))
print("eicu: \t{} \t{}".format(eicu_df_desc.loc['count']['AKI'], eicu_auroc))
print("mimic: \t{} \t{}".format(mimic_df_desc.loc['count']['AKI'], mimic_auroc))